<a href="https://colab.research.google.com/github/GarimaChopra/Generative_AI/blob/main/Final_project_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The objective of the project is to take a dataset and train it to perform a specific task i.e text generation for this project.
For this project we will be training the model for Indian recipes dataset and then use the trained model to generate new recipes for ingredients provided.


In [1]:
#Importing Libraries
import pandas as pd
import numpy as np


In [2]:
#Loading the Dataset
df=pd.read_csv('https://raw.githubusercontent.com/GarimaChopra/Generative_AI/main/Research%20Project/Indian_Food_Cleaned_Dataset1.csv')
df.head(20)

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions
0,Masala Karela Recipe,"salt,amchur (dry mango powder),karela (bitter ...",45,Indian,"To begin making the Masala Karela Recipe,de-se..."
1,Spicy Tomato Rice (Recipe),"tomato,salt,chickpea lentils,green chilli,rice...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato..."
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"salt,rice vermicelli noodles (thin),asafoetida...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi..."
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"tomato,salt,ginger,red chillies,curry,asafoeti...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ..."
5,Pudina Khara Pongal Recipe (Rice and Lentils C...,"cashew nuts,salt,yellow moong dal (split),ging...",30,South Indian Recipes,"To begin making Pudina Khara Pongal Recipe, wa..."
6,Udupi Style Ash Gourd Coconut Curry Recipe,"salt,tamarind water,coconut,mustard seeds,jagg...",40,Udupi,To begin making the Udupi Style Ash Gourd Coco...
7,Mexican Style Black Bean Burrito Recipe,"tomato,spring onion greens,hung curd (greek yo...",40,Mexican,"To begin making the Black Bean Burrito recipe,..."
8,Spicy Crunchy Masala Idli Recipe,"tomato,salt,coriander (dhania) leaves,idli,red...",30,South Indian Recipes,"To prepare Spicy Crunchy Masala Idli Recipe, H..."
9,Cauliflower Leaves Chutney (Recipe in Hindi),"tomato,salt,turmeric powder,red chillies,spoon...",25,South Indian Recipes,"To make cauliflower leaf chutney, first of all..."


In [3]:
df.shape

(5938, 5)

In [4]:
#Dropping columns not needed
columns_to_drop = ['TotalTimeInMins']
df = df.drop(columns = columns_to_drop).dropna()
df.shape

(5938, 4)

In [5]:
df.head(10)

,TranslatedRecipeName,TranslatedIngredients,Cuisine,TranslatedInstructions
0,Masala Karela Recipe,"salt,amchur (dry mango powder),karela (bitter ...",Indian,"To begin making the Masala Karela Recipe,de-se..."
1,Spicy Tomato Rice (Recipe),"tomato,salt,chickpea lentils,green chilli,rice...",South Indian Recipes,"To make tomato puliogere, first cut the tomato..."
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"salt,rice vermicelli noodles (thin),asafoetida...",South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi..."
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",Andhra,To begin making Gongura Chicken Curry Recipe f...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"tomato,salt,ginger,red chillies,curry,asafoeti...",Andhra,"To make Andhra Style Alam Pachadi, first heat ..."
5,Pudina Khara Pongal Recipe (Rice and Lentils C...,"cashew nuts,salt,yellow moong dal (split),ging...",South Indian Recipes,"To begin making Pudina Khara Pongal Recipe, wa..."
6,Udupi Style Ash Gourd Coconut Curry Recipe,"salt,tamarind water,coconut,mustard seeds,jagg...",Udupi,To begin making the Udupi Style Ash Gourd Coco...
7,Mexican Style Black Bean Burrito Recipe,"tomato,spring onion greens,hung curd (greek yo...",Mexican,"To begin making the Black Bean Burrito recipe,..."
8,Spicy Crunchy Masala Idli Recipe,"tomato,salt,coriander (dhania) leaves,idli,red...",South Indian Recipes,"To prepare Spicy Crunchy Masala Idli Recipe, H..."
9,Cauliflower Leaves Chutney (Recipe in Hindi),"tomato,salt,turmeric powder,red chillies,spoon...",South Indian Recipes,"To make cauliflower leaf chutney, first of all..."


In [6]:
 #Dropping irrelevant receipes from other cuisines
cuisines_to_drop = ['Mexican', 'Italian Recipes', 'Thai', 'Chinese', 'Asian', 'Middle Eastern', 'European',
                   'Arab', 'Japanese', 'Vietnamese', 'British', 'Greek', 'French', 'Mediterranean', 'Sri Lankan',
                   'Indonesian', 'African', 'Korean', 'American', 'Carribbean', 'World Breakfast', 'Malaysian', 'Dessert',
                   'Afghan', 'Snack', 'Jewish', 'Brunch', 'Lunch', 'Continental', 'Fusion']

df = df[~df['Cuisine'].isin(cuisines_to_drop)]
df.shape

(4056, 4)

In [7]:
#Cleaning Receipes Name
import re
def clean_recipe(recipe_str):

    # Remove content inside parentheses
    cleaned = re.sub(r'\(.*?\)', '', recipe_str)

    # Extract the string before " - " pattern
    cleaned = cleaned.split(" - ")[0]
    cleaned = cleaned.split(" | ")[0]
    return cleaned.strip()

df["TranslatedRecipeName"] = df["TranslatedRecipeName"].apply(clean_recipe)


<ipython-input-7-ad0d3a04cd3b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TranslatedRecipeName"] = df["TranslatedRecipeName"].apply(clean_recipe)


In [8]:
df.sample(20)

,TranslatedRecipeName,TranslatedIngredients,Cuisine,TranslatedInstructions
900,Suva Methi Baingan Sabzi Recipe,"tomato,salt,ginger,dill leaves,mustard oil,mus...",North Indian Recipes,We begin making the Suva Methi Baingan Sabzi R...
1758,Stuffed Bharwa Tinda Recipe,"tomato,tinda (apple gourd),cumin powder (jeera...",Punjabi,To begin making the Stuffed Bharwa Tinda Recip...
4137,Garlic Kheer,"cardamom powder,milk,rose water,sugar,vinegar ...",Indian,"To make garlic pudding, first put 1 cup of wat..."
432,Chawli Ki Phali & Aloo Sabzi Recipe,"salt,amchur (dry mango powder),fennel seeds (s...",North Indian Recipes,To begin making the Chawli Ki Phali & Aloo Sab...
1377,Khatti Meethi Mirchi Ki Sabzi Recipe,"salt,amchur (dry mango powder),asafoetida (hin...",North Indian Recipes,To begin making the Khatti Meethi Mirchi Ki Sa...
1623,Lau Chingri Recipe,"tomato,salt,cumin powder (jeera),ginger,bay le...",Bengali Recipes,"To begin making the Lau Chingri recipe, marina..."
4459,Zucchini Wheat Semolina Salt Recipe,"salt,wheat (dalia godumai rava),ginger,asafoet...",South Indian Recipes,To begin making the Sundaikkai Godumai Rava Up...
3703,Spicy Broken Wheat Khichdi With Vegetables Recipe,"tomato,wheat (dalia godumai rava),carrot,onion...",Indian,To begin making the Broken Wheat Khichdi With ...
60,Kaddu Palak Roti Recipe,"salt,wheat flour,spinach,red chilli powder,sun...",North Indian Recipes,"To begin making the Kaddu Palak Roti recipe, w..."
2989,Makai Dhokla Recipe,"salt,enos fruit salt,ginger,sunflower oil grea...",Gujarati Recipes﻿,"To begin making the Makai Dhokla, firstly crus..."


In [9]:
df.shape

(4056, 4)

In [60]:
#Cleaning Ingredients
def clean_ingredients(ingredient_str):
    # Split the input ingredient string by commas
    ingredients = ingredient_str.split(',')
    # List of descriptors to remove from ingredient list
    descriptors = [
        'teaspoon', 'tablespoon', 'cup','cups','as per use', 'grams', 'pieces', 'sliced', 'chopped',
        'finely', 'diced', 'desi', 'gms', 'tbsp', 'tsp', 'ml', 'inch', 'large',
        'medium', 'small', 'shredded', 'to taste', 'roughly', 'fresh', 'peeled',
        'de seeded', 'deseeded', 'crushed', 'whole', 'cubes', 'cube', 'round',
        'grated', 'powder', 'optional', 'dry', 'washed', 'soaked', 'cooked',
        'uncooked', 'ripe', 'unripe', 'frozen', 'thin', 'thick', 'cleaned', 'thinly',
        'for', 'into', 'and', 'as per taste', 'to', 'cut', 'overnight', 'leaves',
        'into strips', 'ti','minutes','whisked','salt','to taste','taste','Salt','tablespoons','Red','Green','as','Haldi','Jeera','Dhaniya'
        'oil','or','a','black','dry','masala','deep','Oil','Teaspoon','Leaves','powder','Dry','Whole','fresh',
        'Garam','gram','seeds','Required','water','Water','green','white','per','pinch','paste','slit','&','leaf','boiled',
        'yellow','garnish','curry','minced','mashed','optional','few','fresh','required','in','use','of','sprig','Badi','Kala','red'
    ]

    cleaned_ingredients = []
    for ingredient in ingredients:
        # Remove digits
        ingredient = re.sub(r'\d+', '', ingredient)
        # Replace dashes and slashes with spaces
        ingredient = re.sub(r'[-/]', ' ', ingredient)
        ingredient=re.sub(r"[\([{})\]]", "", ingredient)

        # Remove descriptor words
        ingredient = ' '.join(
            [word for word in ingredient.split() if word not in descriptors]
        )
        cleaned_ingredients.append(ingredient.strip())
    return ', '.join(cleaned_ingredients)

# Fill NA values in 'Cleaned-Ingredients' column
df['TranslatedIngredients'].fillna("", inplace=True)
# Filter out rows containing Devanagari script characters
df = df[~df["TranslatedIngredients"].str.contains(r'[ऀ-ॿ]')]
df["TranslatedIngredients"] = df["TranslatedIngredients"].apply(clean_ingredients)

In [11]:
df.shape

(4056, 4)

In [12]:
#Frequency Distribution for TranslatedIngredients
import nltk
vocabulary = nltk.FreqDist()

for ingredients in df['TranslatedIngredients']:
    ingredients = ingredients.split()
    vocabulary.update(ingredients)
# initialize nltk's lemmatizer
for word, frequency in vocabulary.most_common(200):
    print(f'{word};{frequency}')

,;5294
chilli,;2529
sunflower;1874
oil;1794
cumin;1762
turmeric,;1759
onion,;1695
coriander;1670
jeera,;1653
chillies,;1588
dhania,;1566
garlic,;1422
cloves;1383
ginger,;1300
dal;1259
mustard,;1128
tomato,;1051
oil,;1039
coriander,;851
asafoetida;848
ghee,;847
hing,;829
cardamom;796
coconut,;737
flour;669
sugar,;645
flour,;563
split,;546
dal,;538
lemon,;536
cinnamon;525
methi;522
urad;520
fenugreek,;515
garam,;490
curd,;474
dalchini,;463
elaichi;460
stick;459
milk,;434
rice,;432
wheat;420
potato;411
bay;405
jaggery,;402
tej;399
patta,;397
chana;395
podsseeds,;391
tamarind,;385
pepper,;371
coconut;369
aloo,;353
peppercorns,;341
chillies;333
elaichi,;330
carrot,;326
ginger;321
cashew;309
mango,;301
fennel;300
sesame;298
besan,;297
pepper;293
rice;289
saunf,;285
nuts,;283
mint;279
bengal;277
mustard;273
garam;265
onion;257
moong;251
laung,;246
capsicum,;231
bell;230
ajwain;228
pudina,;226
dried;223
peas;222
carom,;218
turmeric;214
peanuts;206
amchur;202
split;195
kashmiri;194
matar,;193
b

In [13]:
! pip install -U accelerate
! pip install -U transformers

In [14]:
#Checking TranslatedInstruction for Hindi language Charachters
df = df[~df["TranslatedInstructions"].str.contains(r'[ऀ-ॿ]')]

In [15]:
df.shape

(4056, 4)

In [17]:
#Importing Libraries
import torch
import torch.nn as nn
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from tqdm.auto import tqdm

In [16]:
model_name = 'gpt2'

In [18]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
model = GPT2LMHeadModel.from_pretrained(model_name)

# add new, random embeddings for the new tokens
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [19]:
model_save_path = '/content/sample_data/Research'


In [20]:
tokenizer.save_pretrained(model_save_path)

('/content/sample_data/Research/tokenizer_config.json',
 '/content/sample_data/Research/special_tokens_map.json',
 '/content/sample_data/Research/vocab.json',
 '/content/sample_data/Research/merges.txt',
 '/content/sample_data/Research/added_tokens.json',
 '/content/sample_data/Research/tokenizer.json')

In [21]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])

[50259]

In [22]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(inputs,max_length=256,do_sample=True,pad_token_id=50259)
    print(tokenizer.decode(output[0]))

In [23]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>'}

In [24]:
tokenizer.convert_tokens_to_ids(['<|startoftext|>'],)

[50257]

In [25]:
def print_recipe(idx):
    print(f"{df['Ingredients'][idx]}\n\n{df['instructions'][idx]}")

In [26]:
def form_string(ingredient,instruction):
    s = f"<|startoftext|>Ingredients:\n{ingredient.strip()}\n\nInstructions:\n{instruction.strip()}<|endoftext|>"
    return s

In [27]:
data = df.apply(lambda x:form_string(x['TranslatedIngredients'],x['TranslatedInstructions']),axis=1).to_list()

In [28]:
# Split data in training and Validation set
train_size = 0.85
train_len = int(train_size * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

Every tensor passed to the model should be the same length. If the recipe is shorter than 1024 tokens, it will be padded to a length of 1024 using the padding token. In addition, an attention mask will be returned that needs to be passed to the model to tell it to ignore the padding tokens.

In [29]:
#dataset class
class RecipeDataset:
    def __init__(self,data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=1024,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [30]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }

In [31]:
train_ds = RecipeDataset(train_data)
val_ds = RecipeDataset(val_data)

  0%|          | 0/3447 [00:00<?, ?it/s]

  0%|          | 0/609 [00:00<?, ?it/s]

In [32]:
# Training Parameters
args = TrainingArguments(output_dir=model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=3,
                         save_strategy='no'
                        )

In [33]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [34]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim,scheduler)
                 )

In [35]:
trainer.train()

Step,Training Loss
500,1.008700
1000,0.718100
1500,0.672000
2000,0.628900
2500,0.623300


TrainOutput(global_step=2586, training_loss=0.7260052296696756, metrics={'train_runtime': 3448.2474, 'train_samples_per_second': 2.999, 'train_steps_per_second': 0.75, 'total_flos': 5404041805824000.0, 'train_loss': 0.7260052296696756, 'epoch': 3.0})

In [44]:
model_save_path = '/content/sample_data/Research'
print("Saving model to %s" %model_save_path )

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(model_save_path )
tokenizer.save_pretrained(model_save_path )

Saving model to /content/sample_data/Research


('/content/sample_data/Research/tokenizer_config.json',
 '/content/sample_data/Research/special_tokens_map.json',
 '/content/sample_data/Research/vocab.json',
 '/content/sample_data/Research/merges.txt',
 '/content/sample_data/Research/added_tokens.json',
 '/content/sample_data/Research/tokenizer.json')

In [45]:
from transformers import pipeline

In [46]:
model_save_path = '/content/sample_data/Research'
pl = pipeline(task='text-generation',model= '/content/sample_data/Research')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
output_dir = '/content/sample_data/Research'
#Load a trained model and vocabulary that you have fine-tuned
model = GPT2LMHeadModel.from_pretrained(output_dir)

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

In [48]:
def create_prompt(ingredients):
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    return s

In [56]:
ingredients = ['Rice,CHICKEN,tomato']

In [57]:
for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Ingredients:
rice
chicken
tomato
,, ginger, chilli, mustard, cumin jeera, coriander dhania, onion, turmeric, chillies, garam, sunflower oil

Instructions:
To begin making the Karnataka Style Shingu Pachadi Recipe, firstly we have to wash the chicken pieces and keep it aside.Heat oil in a wok/kadai on medium heat.Add the mustard seeds, cumin seeds and allow it to crackle.Add the curry leaves, ginger, green chillies, onion, green chili and saute for a minute.Add the tomatoes, turmeric powder, red chilli powder and coriander powder.
Saute for a minute.Now add the cooked chicken pieces and saute till it becomes tender.
Add the garam masala, salt and mix everything.
Add water and cook for 5 to 7 minutes.
Turn off the heat and garnish with coriander leaves.Serve the Karnataka Style Shingu Pachadi Recipe along with Steamed Rice and Tomato Onion Cucumber Raita for dinner.
